In [1]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model="facebook/m2m100_418M")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/m2m100_418M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/m2m100_418M").to('cuda')

In [13]:
import pandas as pd
import numpy as np
import csv

In [14]:
!nvidia-smi

Sun Apr 14 12:24:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0              28W /  70W |   1967MiB / 15360MiB |     24%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!pip install transformers
!pip install datasets transformers[sentencepiece] sacrebleu
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00


In [15]:
from datasets import load_dataset
import tensorflow as tf
import pandas as pd
import numpy as np

In [16]:
dataset_path = "cfilt/iitb-english-hindi"
train_dataset = load_dataset(dataset_path, split="train[:1000]")

In [17]:
english_sentences = [item['en'] for item in train_dataset['translation']]
hindi_sentences = [item['hi'] for item in train_dataset['translation']]

In [19]:
from timeit import default_timer as timer
start = timer()
translated = []

for index, (eng, hin) in enumerate(zip(english_sentences, hindi_sentences)):
    encoded_en = tokenizer(eng, return_tensors="pt", padding=True, truncation=True).to('cuda')
    generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("hi")).to('cuda')
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    print(f"Index: {index}\nEnglish: {eng}\nHindi (Original): {hin}\nHindi (Translated): {translation}\n")
    translated.append(translation)

end = timer()

Index: 0
English: Give your application an accessibility workout
Hindi (Original): अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
Hindi (Translated): अपने आवेदन को एक पहुंच प्रशिक्षण दें

Index: 1
English: Accerciser Accessibility Explorer
Hindi (Original): एक्सेर्साइसर पहुंचनीयता अन्वेषक
Hindi (Translated): Accessibility Explorer के बारे में टिप्पणियाँ

Index: 2
English: The default plugin layout for the bottom panel
Hindi (Original): निचले पटल के लिए डिफोल्ट प्लग-इन खाका
Hindi (Translated): नीचे पैनल के लिए डिफ़ॉल्ट प्लगइन लेआउट

Index: 3
English: The default plugin layout for the top panel
Hindi (Original): ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
Hindi (Translated): शीर्ष पैनल के लिए डिफ़ॉल्ट प्लगइन लेआउट

Index: 4
English: A list of plugins that are disabled by default
Hindi (Original): उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है
Hindi (Translated): उन प्लगइन्स की सूची जो डिफ़ॉल्ट रूप से अक्षम हैं

Index: 5
English: Highlight duration
Hindi (Original): अवधि को ह

In [20]:
print("Time taken:", end - start)

Time taken: 234.48215884799998


In [21]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
references = [[hin] for hin in hindi_sentences]  # Create reference list of lists
candidates = translated  # Model translations
bleu_score = corpus_bleu(references, candidates)
print(f"BLEU score: {bleu_score:.4f}")

BLEU score: 0.3636
